In [1]:
!pip install selenium
!pip install selenium beautifulsoup4

   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.5 MB 1.3 MB/s eta 0:00:08
   ---------------------------------------- 0.1/9.5 MB 1.1 MB/s eta 0:00:09
    --------------------------------------- 0.1/9.5 MB 1.0 MB/s eta 0:00:10
   - -------------------------------------- 0.3/9.5 MB 1.6 MB/s eta 0:00:06
   - -------------------------------------- 0.5/9.5 MB 2.0 MB/s eta 0:00:05
   -- ------------------------------------- 0.6/9.5 MB 2.3 MB/s eta 0:00:04
   --- ------------------------------------ 0.8/9.5 MB 2.7 MB/s eta 0:00:04
   ---- ----------------------------------- 1.0/9.5 MB 2.9 MB/s eta 0:00:03
   ---- ----------------------------------- 1.2/9.5 MB 2.9 MB/s eta 0:00:03
   ----- ---------------------------------- 1.4/9.5 MB 3.1 MB/s eta 0:00:03
   ------ --------------------------------- 1.6/9.5 MB 3.2 MB/s eta 0:00:03
   ------- -------------------------------- 1.8/9.5 MB 3.4 MB/s eta 0:00:03
   -------- -------

In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pickle
import time
from bs4 import BeautifulSoup

### 1. Iniciamos sesión y guardamos las cookies para evitar volver a iniciar sesión:

In [83]:
# Configura tu navegador (en este caso, Chrome)
driver = webdriver.Chrome()

# Abre la página de Amazon
driver.get('https://www.amazon.es')

# Espera a que el usuario inicie sesión manualmente
input("Inicia sesión y presiona Enter para continuar...")

# Guarda las cookies en un archivo
cookies = driver.get_cookies()  # Obtiene todas las cookies
with open('amazon_cookies.pkl', 'wb') as f:
    pickle.dump(cookies, f)

print("Cookies guardadas correctamente.")

Inicia sesión y presiona Enter para continuar... 


Cookies guardadas correctamente.


### 2. Creamos funciones para ayudarnos en la extracción de las reseñas:

Obtiene las páginas HTML de un las reseñas de un producto, con un número de estrellas concreto.

In [4]:
def get_reviews_html(driver, product_id, star_filter):
    first_page_url = f"https://www.amazon.es/product-reviews/{product_id}/ref=acr_dp_hist_5?ie=UTF8&filterByStar={star_filter}&reviewerType=all_reviews#reviews-filter-bar"

    html_datas = []
    # Pedimos la página
    driver.get(first_page_url)
    time.sleep(5)  # Espera a que cargue la página

    for i in range (0, 11):
        # Guarda el HTML de la página actual
        html_datas.append(driver.page_source)

        try:
            # Esperar a que el botón "Siguiente" esté presente y sea clickeable
            next_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "a-last"))
            )
            
            # Si el botón "Siguiente" está deshabilitado, hemos llegado al final
            if "a-disabled" in next_button.get_attribute("class"):
                print("No hay más páginas de reseñas.")
                break

            # Hacer clic en "Siguiente"
            next_button.click()
            time.sleep(5)  # Esperar a que cargue la nueva página
        except:
            print("No se encontró el botón 'Siguiente' o se llegó al final de las páginas.")
            break  # Salimos del bucle si no se encuentra el botón

    return html_datas

Dada una página HTML, devuelve las reseñas en formato BeautifulSoup.

In [5]:
def get_reviews_from_html(page_html: str) -> BeautifulSoup:
    soup = BeautifulSoup(page_html, "html.parser")
    reviews = soup.find_all("div", {"class": "a-section celwidget"})
    return reviews

Dada una reseña en formato BeautifulSoup, devuelve la reseña en formato texto.

In [7]:
!pip install langdetect

     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ---------------------------------------- 10.2/981.5 kB ? eta -:--:--
     - ----------------------------------- 30.7/981.5 kB 325.1 kB/s eta 0:00:03
     - ----------------------------------- 30.7/981.5 kB 325.1 kB/s eta 0:00:03
     -- ---------------------------------- 61.4/981.5 kB 363.1 kB/s eta 0:00:03
     ---- ------------------------------- 122.9/981.5 kB 552.2 kB/s eta 0:00:02
     --------- -------------------------- 245.8/981.5 kB 942.1 kB/s eta 0:00:01
     ---------- --------------------------- 276.5/981.5 kB 1.0 MB/s eta 0:00:01
     ------------------- ------------------ 501.8/981.5 kB 1.4 MB/s eta 0:00:01
     -------------------------- ----------- 696.3/981.5 kB 1.8 MB/s eta 0:00:01
     ------------------------------- ------ 809.0/981.5 kB 1.8 MB/s eta 0:00:01
     -------------------------------------  962.6/981.5 kB 2

In [8]:
from langdetect import detect

def get_review_text(soup_object: BeautifulSoup) -> str:
    # Extraemos el texto de la reseña
    review_text = soup_object.find(
        "span", {"class": "a-size-base review-text review-text-content"}
    ).get_text()
    
    # Detectamos el idioma de la reseña
    try:
        lang = detect(review_text)
    except:
        lang = 'unknown'  # En caso de que no se pueda detectar el idioma
    
    # Filtramos solo las reseñas en español
    if lang == 'es':
        return review_text.strip()
    else:
        return None  # Si no está en español, devolvemos None (o podemos omitirla)



### 3. Obtenemos los ID-s de los productos que nos interesan sus reseñas:

In [82]:
ids = [
    "B09DPR2LZW"
    "B0B9Y1HTS7"
    "B0CSFRYL7C"
    "B00YXO5UKY"
    "B08BTHXJFN"
    "B09CDPKPK1"
]


### 4. Extraemos las reseñas:

In [75]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pickle
import time

# Configura el navegador
driver = webdriver.Chrome()

# Abre Amazon
driver.get('https://www.amazon.es')

# Cargar cookies si existen
try:
    with open('amazon_cookies.pkl', 'rb') as f:
        cookies = pickle.load(f)
        for cookie in cookies:
            driver.add_cookie(cookie)

    driver.refresh()
    print("Cookies cargadas correctamente.")
except FileNotFoundError:
    print("No se encontraron cookies guardadas. Inicia sesión manualmente.")

stars_mapping = {
    'one_star': 1,
    'two_star': 2,
    'three_star': 3,
    'four_star': 4,
    'five_star': 5,
    }

reviews = {1: [],
           2: [],
           3: [],
           4: [],
           5: []}

star_filters = ['one_star','two_star','three_star','four_star','five_star']
product_id = "B0B9Y1HTS7"

print(f"Extrayendo reseñas del producto.")
for star_filter in star_filters:
    print(f'Extrayendo reseñas del producto de {star_filter} estrellas...')
    # Obtenemos las páginas HTML de interes
    reviews_html = get_reviews_html(driver, product_id, star_filter)
    
    for html in reviews_html:
        soup_reviews = get_reviews_from_html(html)
        
        for soup_review in soup_reviews:
            review_text = get_review_text(soup_review)
            if review_text:  # Solo añade si no es None (es decir, si la reseña está en español)
                reviews[stars_mapping[star_filter]].append(review_text)
        
    
# Cierra el navegador
driver.quit()


Cookies cargadas correctamente.
Extrayendo reseñas del producto.
Extrayendo reseñas del producto de two_star estrellas...
No hay más páginas de reseñas.


In [77]:
# Contamos el número total de elementos en las listas dentro del diccionario
num_elementos_totales = sum(len(v) for v in reviews.values())
num_elementos_totales


27

### 5. Convertimos las reseñas en un DataFrame:

In [78]:
import pandas as pd

# Inicializamos listas vacías para reseñas y estrellas
review_list = []
stars_list = []

# Recorremos el diccionario y agregamos las reseñas y estrellas a las listas
for stars, review_texts in reviews.items():
    for review in review_texts:
        review_list.append(review)
        stars_list.append(stars)

# Creamos el DataFrame
reviews_df = pd.DataFrame({
    'review': review_list,
    'stars': stars_list
})

# Mostrar el DataFrame
print(reviews_df)


                                               review  stars
0                      No puedo decir que es algo wow      2
1   Fue para un regalo, el crío muy contento, pero...      2
2   Lo usamos en mandos tanto de la PS4 como de Xb...      2
3   Funcionan muy bien pero la polipiel que lleva ...      2
4   Son muy vistosos, agradable estéticamente. El ...      2
5   El producto es correcto por el precio, pero ha...      2
6                                   Muy mal microfono      2
7   Muy decepcionada, se notan que son de mala cal...      2
8   Diadema muy pequeña parece que para niños.Para...      2
9   Parecen de plástico de baja calidad, el conect...      2
10  No están mal para el precio que tienen, se esc...      2
11  Porque no se iluminan los cascos, si tienen lu...      2
12  La verdad, compre estos auriculares por las bu...      2
13  Los auriculares como tal suenan bien.Que es lo...      2
14  Los compré y apenas a la semana ya dejó de esc...      2
15  El producto calidad 

### 6. Guardamos el DataFrame en un archivo .csv:

In [79]:
# Guardar el DataFrame en un archivo CSV
reviews_df.to_csv('reviews_cascos_6.csv', index=False, sep = ';')

### 7. Fusionamos los 3 datasets generados:

In [80]:
# Rutas de los datasets
path_1 = "reviews_cascos_1.csv"
path_2 = "reviews_cascos_2.csv"
path_3 = "reviews_cascos_3.csv"
path_4 = "reviews_cascos_4.csv"
path_5 = "reviews_cascos_5.csv"
path_6 = "reviews_cascos_6.csv"

# Cargamos los datasets
data_1 = pd.read_csv(path_1,sep=';', encoding="utf-8")
data_2 = pd.read_csv(path_2,sep=';', encoding="utf-8")
data_3 = pd.read_csv(path_3,sep=';', encoding="utf-8")
data_4 = pd.read_csv(path_4,sep=';', encoding="utf-8")
data_5 = pd.read_csv(path_5,sep=';', encoding="utf-8")
data_6 = pd.read_csv(path_6,sep=';', encoding="utf-8")

# Fusionamos los tres DataFrames
data_combined = pd.concat([data_1, data_2, data_3, data_4, data_5, data_6], ignore_index=True)

print(data_combined.head())
# Imprimir la longitud (número de filas)
print("Número de filas:", len(data_combined))

                                              review  stars
0  Empezaré mi reseña explicando no soy alguien q...      1
1  Increíble como ha llegado la caja. Me parece d...      1
2  Recién compradosLos enchufas a la switch con e...      1
3  Hola, no se encienden las luces hay que hacer ...      1
4  Hace 2 años que se los compré a mi hijo y las ...      1
Número de filas: 1147


In [81]:
# Guardamos el dataset combinado en un nuevo CSV
data_combined.to_csv("reviews_cascos_gaming.csv", index=False, sep=';')